In [1]:
%run common_data_setup.ipynb

[2024-01-05 09:24:14,963 INFO] [numexpr.utils.utils._init_num_threads] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [25]:
%run utils.ipynb

In [4]:
from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

[2024-01-05 09:25:36,139 DEBUG] [matplotlib.__init__.wrapper] matplotlib data path: C:\Users\mikem\anaconda3\Lib\site-packages\matplotlib\mpl-data
[2024-01-05 09:25:36,147 DEBUG] [matplotlib.__init__.wrapper] CONFIGDIR=C:\Users\mikem\.matplotlib
[2024-01-05 09:25:36,149 DEBUG] [matplotlib.__init__.<module>] interactive is False
[2024-01-05 09:25:36,150 DEBUG] [matplotlib.__init__.<module>] platform is win32
[2024-01-05 09:25:36,328 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=C:\Users\mikem\.matplotlib
[2024-01-05 09:25:36,333 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json
[2024-01-05 09:25:36,333 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json


In [4]:
league_ids

{'2015': 853925,
 '2016': 810238,
 '2017': 192551,
 '2018': 799346,
 '2019': 820083,
 '2020': 394682,
 '2021': 364325,
 '2022': 564704,
 '2023': 333146}

In [5]:
game_keys_df

,season,code
16,2015,348
17,2016,359
18,2017,371
19,2018,380
20,2019,390
21,2020,399
22,2021,406
23,2022,414
24,2023,423


In [56]:
#make initial query scoped to individual fantasy football season
yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id=820083,
    game_code='nfl',
    game_id=390
)

#from the query, get teams, managers, and manger ids while cleaning residual string gunk from query
teams = yahoo_query.get_league_teams()
team_ids_list = []
for team in teams:
    team_name = str(team.name)[1:].strip("'").strip('"')
    manager = team.managers[0].nickname
    id = team.team_id
    info = {'team_name': team_name, 'manager_nickname': manager, 'id': id}
    team_ids_list.append(info)

#stash team data in dataframe
team_ids_df = pd.DataFrame(team_ids_list)

In [57]:
# establish the maximum amount of weeks the season ran (via yahoo query method)
max_weeks = len(yahoo_query.get_game_info_by_game_id(348))

# iterate through each week, then each manager by id, and generate roster points for each
season_stats = []
print("Progress through week: ")
for week in range(1, max_weeks+1):
    print(f"|{week}|", end=" ")
    for manager_id in team_ids_df['id']:
        # raw player data by manager, week comes from query method
        try: 
            week_subquery = yahoo_query.get_team_roster_player_stats_by_week(manager_id, week)
        except:
            print(f"Yahoo Error occurred when making call for manager ID {manager_id}, week {week}")
        else:
            # and is passed to custom method that organizes it into data we care about
            weekly_stats = generate_roster_points_by_week(week_subquery)
    
            # before adding the row, add the team, manager, and week inf
            team_name = team_ids_df.loc[team_ids_df['id'] == manager_id, 'team_name'].item()
            weekly_stats['team_name'] = team_name
            weekly_stats['manager'] = map_teams_to_name(team_name)
            weekly_stats['week'] = week
            season_stats.append(weekly_stats)

#stash in a dataframe
season_stats_df = pd.DataFrame(season_stats)

Progress through week: 
|1| |2| |3| |4| |5| |6| |7| |8| |9| |10| |11| |12| |13| |14| |15| 

/c/Users/mikem/yahoo/yahoo-app


In [61]:
season_stats_df.to_csv('2019_weekly_roster_stats.csv', index=False)